In [1]:
import pandas as pd
import csv

dataset = pd.read_csv("dataset_eurousd.csv",sep=None)

/opt/conda/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [2]:
dataset

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>
0,2012.04.09,00:00:00,1.30961,1.30965,1.30662,1.30687,2151,0,13
1,2012.04.09,01:00:00,1.30688,1.30861,1.30668,1.30847,1490,0,12
2,2012.04.09,02:00:00,1.30848,1.30876,1.30441,1.30443,2226,0,12
3,2012.04.09,03:00:00,1.30444,1.30524,1.30330,1.30508,2322,0,13
4,2012.04.09,04:00:00,1.30509,1.30618,1.30504,1.30581,1379,0,12
...,...,...,...,...,...,...,...,...,...
61997,2022.04.08,14:00:00,1.08861,1.08881,1.08592,1.08638,3897,0,0
61998,2022.04.08,15:00:00,1.08637,1.08660,1.08504,1.08531,4443,0,0
61999,2022.04.08,16:00:00,1.08531,1.08581,1.08364,1.08399,6250,0,0
62000,2022.04.08,17:00:00,1.08399,1.08804,1.08388,1.08727,6825,0,0


In [3]:
dataset.shape

(62002, 9)

In [4]:
dataset.head()

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>
0,2012.04.09,00:00:00,1.30961,1.30965,1.30662,1.30687,2151,0,13
1,2012.04.09,01:00:00,1.30688,1.30861,1.30668,1.30847,1490,0,12
2,2012.04.09,02:00:00,1.30848,1.30876,1.30441,1.30443,2226,0,12
3,2012.04.09,03:00:00,1.30444,1.30524,1.30330,1.30508,2322,0,13
4,2012.04.09,04:00:00,1.30509,1.30618,1.30504,1.30581,1379,0,12


In [5]:
dataset.isnull().sum()

<DATE>       0
<TIME>       0
<OPEN>       0
<HIGH>       0
<LOW>        0
<CLOSE>      0
<TICKVOL>    0
<VOL>        0
<SPREAD>     0
dtype: int64

In [6]:
from sklearn.preprocessing import MinMaxScaler

price = dataset[['<CLOSE>']]
print(price)
scaler = MinMaxScaler(feature_range=(-1, 1))
price['<CLOSE>'] = scaler.fit_transform(price['<CLOSE>'].values.reshape(-1,1))
print(price)


       <CLOSE>
0      1.30687
1      1.30847
2      1.30443
3      1.30508
4      1.30581
...        ...
61997  1.08638
61998  1.08531
61999  1.08399
62000  1.08727
62001  1.08757

[62002 rows x 1 columns]
        <CLOSE>
0      0.504228
1      0.513098
2      0.490699
3      0.494303
4      0.498351
...         ...
61997 -0.718238
61998 -0.724170
61999 -0.731489
62000 -0.713304
62001 -0.711640

[62002 rows x 1 columns]


/tmp/ipykernel_114/512202299.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['<CLOSE>'] = scaler.fit_transform(price['<CLOSE>'].values.reshape(-1,1))


In [7]:
import numpy as np

def split_data(stock, lookback):
    print(stock)
    data_raw = stock.to_numpy() # convert to numpy array
    data = []
    print("data raw ==>", data_raw)
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]

lookback = 20 # choose sequence length
x_train, y_train, x_test, y_test = split_data(price, lookback)

        <CLOSE>
0      0.504228
1      0.513098
2      0.490699
3      0.494303
4      0.498351
...         ...
61997 -0.718238
61998 -0.724170
61999 -0.731489
62000 -0.713304
62001 -0.711640

[62002 rows x 1 columns]
data raw ==> [[ 0.50422754]
 [ 0.51309844]
 [ 0.49069942]
 ...
 [-0.73148893]
 [-0.71330358]
 [-0.71164028]]


In [8]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(49586, 19, 1)
(49586, 1)
(12396, 19, 1)
(12396, 1)


In [9]:
import torch
import torch.nn as nn
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
y_train_gru = torch.from_numpy(y_train).type(torch.Tensor)
y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)

In [10]:
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 100

In [11]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [12]:
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
import time
hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []
for t in range(num_epochs):
    y_train_pred = model(x_train)
    lstm.append(y_train_pred)
    loss = criterion(y_train_pred, y_train_lstm)
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    
training_time = time.time()-start_time
print("Training time: {}".format(training_time))

Epoch  0 MSE:  0.38115179538726807
Epoch  1 MSE:  0.29650160670280457
Epoch  2 MSE:  0.2413964420557022
Epoch  3 MSE:  0.22804899513721466
Epoch  4 MSE:  0.16061094403266907
Epoch  5 MSE:  0.0982905924320221
Epoch  6 MSE:  0.045901499688625336
Epoch  7 MSE:  0.054936595261096954
Epoch  8 MSE:  0.05624418705701828
Epoch  9 MSE:  0.05639388784766197
Epoch  10 MSE:  0.0353931300342083
Epoch  11 MSE:  0.036020923405885696
Epoch  12 MSE:  0.016346724703907967
Epoch  13 MSE:  0.00848097912967205
Epoch  14 MSE:  0.011482937261462212
Epoch  15 MSE:  0.013208402320742607
Epoch  16 MSE:  0.011141662485897541
Epoch  17 MSE:  0.007964245975017548
Epoch  18 MSE:  0.006755116395652294
Epoch  19 MSE:  0.008382577449083328
Epoch  20 MSE:  0.011180425062775612
Epoch  21 MSE:  0.012125941924750805
Epoch  22 MSE:  0.010142386890947819
Epoch  23 MSE:  0.00815874058753252
Epoch  24 MSE:  0.008215653710067272
Epoch  25 MSE:  0.009146437980234623
Epoch  26 MSE:  0.009061270393431187
Epoch  27 MSE:  0.0074294

In [ ]:
from matplotlib import pyplot as plt

plt.plot(hist)
plt.title('Training Loss per epoch')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

plt.plot(y_train_lstm, label="Data")
# plt.plot(y_train_pred, label="Training Prediction(LSTM)")
plt.ylabel('Closing Price')
plt.xlabel('Hour')
plt.title('EURUSD closing price')
plt.show()



In [15]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62002 entries, 0 to 62001
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   <DATE>     62002 non-null  object 
 1   <TIME>     62002 non-null  object 
 2   <OPEN>     62002 non-null  float64
 3   <HIGH>     62002 non-null  float64
 4   <LOW>      62002 non-null  float64
 5   <CLOSE>    62002 non-null  float64
 6   <TICKVOL>  62002 non-null  int64  
 7   <VOL>      62002 non-null  int64  
 8   <SPREAD>   62002 non-null  int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 4.3+ MB
